In [209]:
import sys,os
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from IPython.display import display

#os.chdir("/content")
#from colorsetup import colors, palette
#sns.set_palette(palette)

# ignore warnings
warnings.filterwarnings('ignore')

%matplotlib inline
plotsize = (13, 5)
plt.rcParams['figure.figsize'] = plotsize

In [210]:
from matplotlib.pyplot import subplots
from sklearn.linear_model import \
     (LinearRegression,
      LogisticRegression,
      Lasso)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from ISLP import load_data
from ISLP.models import ModelSpec as MS
from sklearn.model_selection import \
     (train_test_split,
      GridSearchCV)

import torch
from torch import nn
from torch.optim import RMSprop
from torch.utils.data import TensorDataset

from torchmetrics import (MeanAbsoluteError,
                          R2Score)
from torchinfo import summary

from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CSVLogger

In [211]:
from ISLP.torch import (SimpleDataModule,
                        SimpleModule,
                        ErrorTracker,
                        rec_num_workers)

In [212]:
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (#ModelSpec as MS,
                         summarize) # the enclosing round parentheses is to write codes in multiple lines. 
from ISLP import confusion_table
from ISLP.models import contrast
from sklearn.discriminant_analysis import \
     (LinearDiscriminantAnalysis as LDA,
      QuadraticDiscriminantAnalysis as QDA)
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
# 

In [213]:
## Obtain the data and write it to a csv file. Run it only once
# apple = yf.Ticker("AAPL")
# apple_share_price_data = apple.history(period="max")
# file_path = 'apple_20240301.csv'
# apple_share_price_data.to_csv(file_path)

In [214]:
def read_data(csv_file='apple_df.csv',dc='Date', verbose = False):
  """
  Read a csv file into a dataframe and convert the date string into a date index
  INPUT: 
  csv_file: complete csv_file path including file name
  dc: Date column name
  OUTPUT: 
  df: a dataFrame with date value as index
  """
  df =pd.read_csv(csv_file)
  
  df[dc]=df[dc].apply(lambda x: x[:11])
  df.set_index(dc, inplace=True)
  df.index= pd.DatetimeIndex(df.index)
  if verbose: 
    print(df.head())
    print("\n")
    print("*"*20+"DataFrame Info"+"*"*20)
    df.info()
    print("\n")
    print(df.describe().transpose())
  return df

In [215]:

file_path = 'apple_20240301.csv'
apple = read_data('apple_20240301.csv')
apple.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10895 entries, 1980-12-12 to 2024-03-01
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          10895 non-null  float64
 1   High          10895 non-null  float64
 2   Low           10895 non-null  float64
 3   Close         10895 non-null  float64
 4   Volume        10895 non-null  int64  
 5   Dividends     10895 non-null  float64
 6   Stock Splits  10895 non-null  float64
dtypes: float64(6), int64(1)
memory usage: 680.9 KB


In [216]:
### remove the rows with Volume =0, on which there were no transaction 
apple = apple[apple['Volume'] != 0]

In [217]:
apple.drop(['Dividends','Stock Splits'], axis=1, inplace=True)
apple['month'] = apple.index.month
apple['month'] = apple['month'].astype('category')

apple['day_of_week'] = apple.index.day_name()
apple['day_of_week'] = apple['day_of_week'].astype('category')
day_order = [ 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
apple['day_of_week'] = apple['day_of_week'].cat.set_categories(day_order)

apple['return']=np.log(apple.Close/(apple.Close.shift(1)))
apple['volatility'] = np.fabs(apple['return'])

window = 100
apple['log_volume'] = np.log(apple['Volume']/apple['Volume'].rolling(window).mean().shift(1)) 
#rolling creates 99 Nan, shift creates 1 nan: togeter 100 nan

direction_threshold = 0 
apple['direction'] = apple['return'] > direction_threshold

apple.dropna(inplace=True)

apple['train'] =  ~(apple.index >= '2022-12-25' )  #this choice is to make sure that the last batch, 
# when using batch size 64, has at least two examples. The last batch must have at least two examples 
# to computer R^2  when using Torch NN. 

In [219]:
#### Normalize the columns
cols = ['return', 'log_volume', 'volatility']
X = pd.DataFrame(StandardScaler(
                     with_mean=True,
                     with_std=True).fit_transform(apple[cols]),
                 columns=apple[cols].columns,
                 index=apple.index)

In [220]:
# Calculate the lag columns
max_lag = 5
for lag in range(1, max_lag+1):
    for col in cols:
        newcol = np.zeros(X.shape[0]) * np.nan
        newcol[lag:] = X[col].values[:-lag]
        X.insert(len(X.columns), "{0}_{1}".format(col, lag), newcol)#insert at the end
X.insert(len(X.columns), 'train', apple['train']) #insert the col training identifier 

In [221]:
X = X.dropna() #drop max_lag=5 rows with nan

In [223]:
train_len, test_len = len(X[X['train']==True]), len(X[X['train']!=True])
train_len, test_len

(10493, 296)

## Logistic Regression using statsmodels

In [230]:
X_train, X_test = X.loc[X['train']], X.loc[~X['train']] # X[X['train'] == True] eqiv to  X.loc[X['train']]
Y_train, Y_test = X_train['return']>0, X_test['return']>0

In [231]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((10493, 19), (296, 19), (10493,), (296,))

In [232]:
predictors = ['return_5', 'return_4','return_3', 'return_2', 'return_1', 'log_volume_1']
design = MS(predictors)
X_train_5 = design.fit_transform(X_train)
y = Y_train
glm = sm.GLM(y,
             X_train_5,
             family=sm.families.Binomial())
results = glm.fit()
summarize(results)

,coef,std err,z,P>|z|
intercept,-0.0495,0.020,-2.530,0.011
return_5,-0.0083,0.019,-0.429,0.668
return_4,0.0508,0.020,2.593,0.010
return_3,-0.0388,0.020,-1.979,0.048
return_2,-0.0680,0.020,-3.444,0.001
return_1,-0.0098,0.019,-0.507,0.612
log_volume_1,0.0203,0.019,1.046,0.296


In [233]:
X_test_5 = design.transform(X_test)
probs = results.predict(exog= X_test_5)

labels = probs > 0.5
confusion_table(labels, Y_test)

Truth,False,True
Predicted,,
False,133,131
True,14,18


In [234]:
### Test Accuracy
np.mean(labels == Y_test)


0.5101351351351351

In [235]:
# Using two lags
predictors = [ 'return_2', 'return_1']
design = MS(predictors)
X_train_2 = design.fit_transform(X_train)
y = Y_train
glm = sm.GLM(y,
             X_train_2,
             family=sm.families.Binomial())
results = glm.fit()
print(summarize(results), '\n')
X_test_2 = design.transform(X_test)
probs = results.predict(exog= X_test_2)

labels = probs > 0.5
print("Confusion Table: \n", confusion_table(labels, Y_test) )
### Test Accuracy
print("Test Accuracy: ", np.mean(labels == Y_test) )

             coef  std err      z  P>|z|
intercept -0.0494    0.020 -2.529  0.011
return_2  -0.0694    0.020 -3.515  0.000
return_1  -0.0101    0.019 -0.522  0.601 

Confusion Table: 
 Truth      False  True 
Predicted              
False        144    139
True           3     10
Test Accuracy:  0.5202702702702703


## Logistic Regression using SK-learn

In [236]:
X_train_2, X_test_2 = [M.drop(columns=['intercept']) #sklearn automatically adds an intercept, drop it from the design matrix by statsmodels
                   for M in [X_train_2, X_test_2]]

In [237]:
logit = LogisticRegression(C=1e10, solver='liblinear')
logit.fit(X_train_2, Y_train)
logit_pred = logit.predict_proba(X_test_2)
logit_labels = np.where(logit_pred[:,1] > .5, True, False)
confusion_table(logit_labels, Y_test)

Truth,False,True
Predicted,,
False,144,139
True,3,10


In [238]:
# Logistic accuracy
np.mean(logit_labels == Y_test)

0.5202702702702703

## LDA using Sk-learn

In [178]:
lda = LDA(store_covariance=True) #store the covariance of each class

lda.fit(X_train_2, Y_train) # note sklearn use the order X,Y, opposite the order used by statsmodels: Y, X

LinearDiscriminantAnalysis(store_covariance=True)

In [179]:
lda.means_ #array-like of shape (n_classes, n_features)

array([[ 0.03383844,  0.0050342 ],
       [-0.03609458, -0.0059984 ]])

In [181]:
lda.classes_, lda.priors_

(array([False,  True]), array([0.51234156, 0.48765844]))

In [182]:
lda.scalings_ #Scaling of the features in the space spanned by the class centroids. 
# Only available for ‘svd’ and ‘eigen’ solvers.

array([[-0.97727746],
       [-0.14330167]])

These values provide the linear combination of `Lag1`  and `Lag2`  that are used to form the LDA decision rule (boundary). In other words, these are the multipliers of the elements of $X=x$ in (4.24).
  If $-0.977\times `Lag1`  - 0.143 \times `Lag2` $ is large, then the LDA classifier will predict a market increase, and if it is small, then the LDA classifier will predict a market decline. 

In [183]:
lda_pred = lda.predict(X_test_2)
confusion_table(lda_pred, Y_test)

Truth,False,True
Predicted,,
False,144,139
True,3,10


In [191]:
# LDA accuracy
np.mean(lda_pred == Y_test)

0.5202702702702703

In [185]:
lda_prob = lda.predict_proba(X_test_2) #ndarray of shape (n_samples, n_classes)
np.all(
       np.where(lda_prob[:,1] >= 0.5, True, False) == lda_pred
       )

True

## QDA using Sk-Learn

In [186]:
qda = QDA(store_covariance=True)
qda.fit(X_train_2, Y_train)

QuadraticDiscriminantAnalysis(store_covariance=True)

In [187]:
qda.means_, qda.priors_

(array([[ 0.03383844,  0.0050342 ],
        [-0.03609458, -0.0059984 ]]),
 array([0.51234156, 0.48765844]))

In [188]:
qda.covariance_[0], qda.covariance_[1] #Covariance of each class

(array([[ 1.10241163, -0.00317939],
        [-0.00317939,  1.06730139]]),
 array([[0.93688266, 0.02681325],
        [0.02681325, 0.97636331]]))

In [189]:
qda_pred = qda.predict(X_test_2)
confusion_table(qda_pred, Y_test)

Truth,False,True
Predicted,,
False,13,21
True,134,128


In [190]:
# QDA Accuracy
np.mean(qda_pred == Y_test)  # much worse than LDA

0.47635135135135137

## Naive Bayes

In [192]:
NB = GaussianNB()
NB.fit(X_train_2, Y_train)

GaussianNB()

In [193]:
NB.classes_, NB.class_prior_, 

(array([False,  True]), array([0.51234156, 0.48765844]))

In [194]:
NB.theta_  # class mean, 

array([[ 0.03383844,  0.0050342 ],
       [-0.03609458, -0.0059984 ]])

In [195]:
NB.var_ # Class variance

array([[1.10220657, 1.06710286],
       [0.93669957, 0.97617251]])

In [196]:
nb_labels = NB.predict(X_test_2)
confusion_table(nb_labels, Y_test)

Truth,False,True
Predicted,,
False,12,18
True,135,131


In [197]:
# NB Accuracy
np.mean(nb_labels == Y_test)  # much worse than LDA

0.4831081081081081

## KNN

In [198]:
knn1 = KNeighborsClassifier(n_neighbors=1)
X_train_2, X_test_2 = [np.asarray(X) for X in [X_train_2, X_test_2]]
knn1.fit(X_train_2, Y_train)
knn1_pred = knn1.predict(X_test_2)
confusion_table(knn1_pred, Y_test)

Truth,False,True
Predicted,,
False,79,79
True,68,70


In [199]:
# Accuracy
np.mean(knn1_pred == Y_test)

0.5033783783783784

In [200]:
knn3 = KNeighborsClassifier(n_neighbors=3)
X_train_2, X_test_2 = [np.asarray(X) for X in [X_train_2, X_test_2]]
knn3.fit(X_train_2, Y_train)
knn3_pred = knn3.predict(X_test_2)
confusion_table(knn1_pred, Y_test)

Truth,False,True
Predicted,,
False,79,79
True,68,70


In [202]:
np.mean(knn3_pred == Y_test)

0.4594594594594595

In [204]:
### Tuning the KNN parameters
for K in range(1,6):
    knn = KNeighborsClassifier(n_neighbors=K)
    knn_pred = knn.fit(X_train_2, Y_train).predict(X_test_2)
    C = confusion_table(knn_pred, Y_test)
    templ = ('K={0:d}: # predicted to Go up: {1:>2},' +  # > for right alighment
            '  # which did Go up {2:d}, accuracy {3:.1%}')
    pred = C.loc[True].sum()
    did_rent = C.loc[True,False]
    print(templ.format(
          K,
          pred,
          did_rent,
          did_rent / pred))


K=1: # predicted to Go up: 138,  # which did Go up 68, accuracy 49.3%
K=2: # predicted to Go up: 61,  # which did Go up 29, accuracy 47.5%
K=3: # predicted to Go up: 119,  # which did Go up 65, accuracy 54.6%
K=4: # predicted to Go up: 71,  # which did Go up 42, accuracy 59.2%
K=5: # predicted to Go up: 120,  # which did Go up 65, accuracy 54.2%


## Linear  AR Model

In [ ]:
Y, train = X['log_volume'], X['train']
X = X.drop(columns=['train'] + cols)
# X.columns

In [82]:
M = LinearRegression()
M.fit(X[train], Y[train])
M.score(X[~train], Y[~train])

0.19997338230088157

In [83]:
### Adding `day_of_week`
X_day = pd.merge(X, 
                 pd.get_dummies(apple['day_of_week']),
                 on='Date')

In [84]:
M.fit(X_day[train], Y[train])
M.score(X_day[~train], Y[~train])

0.17872110527570284

### RNN Model

In [85]:
# Reshaping the data
ordered_cols = []
for lag in range(5,0,-1):
    for col in cols:
        ordered_cols.append('{0}_{1}'.format(col, lag))
X = X.reindex(columns=ordered_cols)
# X.columns

In [87]:
X_rnn = X.to_numpy().reshape((-1,5,3))
# X_rnn.shape

In [88]:
class NYSEModel(nn.Module):
    def __init__(self):
        super(NYSEModel, self).__init__()
        self.rnn = nn.RNN(3, #number of features for each time step
                          12, #hidden size
                          batch_first=True) #batch size as the first dim
        self.dense = nn.Linear(12, 1)
        self.dropout = nn.Dropout(0.1)
    def forward(self, x):
        val, h_n = self.rnn(x)
        val = self.dense(self.dropout(val[:,-1]))
        return torch.flatten(val)
nyse_model = NYSEModel()

In [89]:
datasets = []
for mask in [train, ~train]:
    X_rnn_t = torch.tensor(X_rnn[mask].astype(np.float32))
    Y_t = torch.tensor(Y[mask].astype(np.float32))
    datasets.append(TensorDataset(X_rnn_t, Y_t))
nyse_train, nyse_test = datasets

In [90]:
len(nyse_train), len(nyse_test)

(10493, 296)

In [91]:
nyse_test.tensors[0].shape, nyse_test.tensors[1].shape

(torch.Size([296, 5, 3]), torch.Size([296]))

In [92]:
summary(nyse_model,
        input_data=X_rnn_t,
        col_names=['input_size',
                   'output_size',
                   'num_params'])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
NYSEModel                                [296, 5, 3]               [296]                     --
├─RNN: 1-1                               [296, 5, 3]               [296, 5, 12]              204
├─Dropout: 1-2                           [296, 12]                 [296, 12]                 --
├─Linear: 1-3                            [296, 12]                 [296, 1]                  13
Total params: 217
Trainable params: 217
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.31
Input size (MB): 0.02
Forward/backward pass size (MB): 0.14
Params size (MB): 0.00
Estimated Total Size (MB): 0.16

In [93]:
max_num_workers = rec_num_workers()

In [95]:
nyse_dm = SimpleDataModule(nyse_train,
                           nyse_test,
                           num_workers=min(4, max_num_workers),
                           validation=nyse_test,
                           batch_size=64)
                           

In [96]:
# Example the minibatch size of y and the output size in each minibatch
for idx, (x, y) in enumerate(nyse_dm.train_dataloader()):
    out = nyse_model(x)
    print(y.size(), out.size())
    if idx >= 2:
        break

torch.Size([64]) torch.Size([64])
torch.Size([64]) torch.Size([64])
torch.Size([64]) torch.Size([64])


In [97]:
nyse_optimizer = RMSprop(nyse_model.parameters(),
                         lr=0.001)
nyse_module = SimpleModule.regression(nyse_model,
                                      optimizer=nyse_optimizer,
                                      metrics={'r2':R2Score()})

In [98]:
nyse_trainer = Trainer(deterministic=False,
                       max_epochs=20,
                       callbacks=[ErrorTracker()])
nyse_trainer.fit(nyse_module,
                 datamodule=nyse_dm)
nyse_trainer.test(nyse_module,
                  datamodule=nyse_dm)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type      | Params
------------------------------------
0 | model | NYSEModel | 217   
1 | loss  | MSELoss   | 0     
------------------------------------
217       Trainable params
0         Non-trainable params
217       Total params
0.001     Total estimated model params size (MB)


Epoch 199: 100%|██████████| 164/164 [00:01<00:00, 159.77it/s, v_num=14]     

`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 199: 100%|██████████| 164/164 [00:01<00:00, 156.04it/s, v_num=14]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 5/5 [00:00<00:00, 274.37it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Runningstage.testing metric      DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.22541670501232147
         test_r2            0.1901509165763855
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.22541670501232147, 'test_r2': 0.1901509165763855}]

### Build a linear and non-linear AR model using Torch 

In [99]:
datasets = []
for mask in [train, ~train]:
    X_day_t = torch.tensor(
                   np.asarray(X_day[mask]).astype(np.float32))
    Y_t = torch.tensor(np.asarray(Y[mask]).astype(np.float32))
    datasets.append(TensorDataset(X_day_t, Y_t))
day_train, day_test = datasets

In [100]:
# day_train.tensors[0].shape

torch.Size([10493, 20])

In [101]:
day_dm = SimpleDataModule(day_train,
                          day_test,
                          num_workers=min(4, max_num_workers),
                          validation=day_test,
                          batch_size=64)

In [102]:
class NonLinearARModel(nn.Module):
    def __init__(self):
        super(NonLinearARModel, self).__init__()
        self._forward = nn.Sequential(nn.Flatten(), #flatten a multi-dim tensor into a 1-d tensor while keeping the batch size
                                      nn.Linear(20, 32), 
                                      nn.ReLU(),
                                      nn.Dropout(0.5),
                                      nn.Linear(32, 1))
    def forward(self, x):
        return torch.flatten(self._forward(x))


In [103]:
nl_model = NonLinearARModel()
nl_optimizer = RMSprop(nl_model.parameters(),
                           lr=0.001)
nl_module = SimpleModule.regression(nl_model,
                                        optimizer=nl_optimizer,
                                        metrics={'r2':R2Score()})

In [104]:
nl_trainer = Trainer(deterministic=False,
                         max_epochs=20,
                         callbacks=[ErrorTracker()])
nl_trainer.fit(nl_module, datamodule=day_dm)
nl_trainer.test(nl_module, datamodule=day_dm) 

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | NonLinearARModel | 705   
1 | loss  | MSELoss          | 0     
-------------------------------------------
705       Trainable params
0         Non-trainable params
705       Total params
0.003     Total estimated model params size (MB)


Epoch 19: 100%|██████████| 164/164 [00:00<00:00, 186.73it/s, v_num=15]

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 164/164 [00:00<00:00, 186.73it/s, v_num=15]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 5/5 [00:00<00:00, 268.21it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Runningstage.testing metric      DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.21885068714618683
         test_r2            0.21374046802520752
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.21885068714618683, 'test_r2': 0.21374046802520752}]

In [105]:
class LinearARModel(nn.Module):
    def __init__(self):
        super(LinearARModel, self).__init__()
        self._forward = nn.Sequential(nn.Flatten(), #flatten a multi-dim tensor into a 1-d tensor while keeping the batch size
                                    #   nn.Linear(20, 32), 
                                    #   nn.ReLU(),
                                    #   nn.Dropout(0.5),
                                      nn.Linear(20, 1))
    def forward(self, x):
        return torch.flatten(self._forward(x))


In [108]:
linearAR_model = LinearARModel()
linearAR_optimizer = RMSprop(linearAR_model.parameters(),
                           lr=0.001)
linearAR_module = SimpleModule.regression(linearAR_model,
                                        optimizer=linearAR_optimizer,
                                        metrics={'r2':R2Score()})

In [109]:
linearAR_trainer = Trainer(deterministic=False,
                         max_epochs=20,
                         callbacks=[ErrorTracker()])
linearAR_trainer.fit(linearAR_module, datamodule=day_dm)
linearAR_trainer.test(linearAR_module, datamodule=day_dm) 

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type          | Params
----------------------------------------
0 | model | LinearARModel | 21    
1 | loss  | MSELoss       | 0     
----------------------------------------
21        Trainable params
0         Non-trainable params
21        Total params
0.000     Total estimated model params size (MB)


Epoch 19: 100%|██████████| 164/164 [00:00<00:00, 215.59it/s, v_num=16]      

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 164/164 [00:00<00:00, 213.48it/s, v_num=16]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 5/5 [00:00<00:00, 218.06it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Runningstage.testing metric      DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.22817173600196838
         test_r2            0.18025296926498413
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.22817173600196838, 'test_r2': 0.18025296926498413}]